<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/46_local_rf_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [28]:
# !pip install geemap scikit-learn

# How to use locally trained machine learning models with GEE

This notebook illustrates how to train a random forest (or any other ensemble tree estimator) locally using scikit-learn, convert the estimator into a string representation that Earth Engine can interpret, and how to apply the machine learning model with EE. **The notebook and the geemap machine learning module ([ml.py](https://geemap.org/ml/)) were contributed by [Kel Markert](https://github.com/KMarkert). A huge thank you to him.** 

In [29]:
import ee
import geemap
import pandas as pd

from geemap import ml
from sklearn import ensemble

In [30]:
geemap.ee_initialize()

## Train a model locally using scikit-learn

In this demo, we are going to use the training data from [here](https://github.com/giswqs/geemap/blob/master/examples/data/rf_example.csv). 

In [31]:
# read the feature table to train our RandomForest model
# data taken from ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

url = "https://raw.githubusercontent.com/giswqs/geemap/master/examples/data/rf_example.csv"
df = pd.read_csv(url)

In [32]:
df

,B2,B3,B4,B5,B6,B7,landcover
0,0.139846,0.114738,0.109982,0.119542,0.125795,0.105720,0
1,0.130316,0.109207,0.107499,0.140210,0.132006,0.108497,0
2,0.146690,0.135766,0.146550,0.225686,0.218105,0.167111,0
3,0.119413,0.108924,0.105196,0.144868,0.159775,0.122056,0
4,0.155492,0.139932,0.137486,0.151377,0.153771,0.133134,0
...,...,...,...,...,...,...,...
93,0.117331,0.092176,0.062548,0.020362,0.005813,0.004047,2
94,0.118353,0.093785,0.060253,0.020083,0.007317,0.004719,2
95,0.123362,0.095831,0.069663,0.027320,0.011386,0.008357,2
96,0.122907,0.100083,0.079527,0.024564,0.008570,0.006321,2


In [33]:
# specify the names of the features (i.e. band names) and label
# feature names used to extract out features and define what bands

feature_names = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
label = "landcover"

In [34]:
# get the features and labels into seperate variables
X = df[feature_names]
y = df[label]

In [35]:
# create a classifier and fit
n_trees = 100
rf = ensemble.RandomForestClassifier(n_trees).fit(X,y)

## Convert a sklearn classifier object to a list of strings

In [36]:
# convert the estimator into a list of strings
# this function also works with the ensemble.ExtraTrees estimator
trees =  ml.rf_to_strings(rf,feature_names)

In [37]:
# print the first tree to see the result
print(trees[0])

1) root 66 9999 9999 (1.2754884580538501)
  2) B5 <= 0.064203 21 0.0000 2.000000 *
  3) B5 > 0.064203 66 0.6658 1.000000
    6) B2 <= 0.116759 23 0.0000 1.000000 *
    7) B2 > 0.116759 45 0.4990 1.000000
      14) B5 <= 0.291370 21 0.0000 0.000000 *
      15) B5 > 0.291370 1 0.0000 1.000000 *



In [38]:
print(trees[1])

1) root 61 9999 9999 (2.3725881007803515)
  2) B7 <= 0.019032 20 0.0000 2.000000 *
  3) B7 > 0.019032 61 0.6474 2.000000
    6) B6 <= 0.217125 41 0.4919 1.000000
      12) B7 <= 0.100071 32 0.4140 1.000000
        24) B4 <= 0.099514 16 0.0000 1.000000 *
        25) B4 > 0.099514 1 0.0000 0.000000 *
      13) B7 > 0.100071 32 0.4140 1.000000
        26) B3 <= 0.100830 6 0.0000 1.000000 *
        27) B3 > 0.100830 9 0.0000 0.000000 *
    7) B6 > 0.217125 41 0.4919 1.000000
      14) B2 <= 0.134185 2 0.0000 1.000000 *
      15) B2 > 0.134185 7 0.0000 0.000000 *



In [39]:
# number of trees we converted should equal the number of trees we defined for the model
len(trees) == n_trees

True

## Convert sklearn classifier to GEE classifier

At this point you can take the list of strings and save them locally to avoid training again. However, we want to use the model with EE so we need to create an ee.Classifier and persist the data on ee for best results.

In [40]:
# create a ee classifier to use with ee objects from the trees
ee_classifier = ml.strings_to_classifier(trees)

In [41]:
# ee_classifier.getInfo()

## Classify image using GEE classifier

In [42]:
# Make a cloud-free Landsat 8 TOA composite (from raw imagery).
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1');

image = ee.Algorithms.Landsat.simpleComposite(
  collection= l8.filterDate('2018-01-01', '2018-12-31'),
  asFloat= True
)

In [43]:
# classify the image using the classifier we created from the local training
# note: here we select the feature_names from the image that way the classifier knows which bands to use
classified = image.select(feature_names).classify(ee_classifier)

In [44]:
# display results
Map = geemap.Map(center=(37.75,-122.25), zoom=11)

Map.addLayer(image,{"bands": ['B7', 'B5', 'B3'], "min":0.05, "max": 0.55, "gamma":1.5}, 'image')
Map.addLayer(classified, {"min": 0, "max": 2, "palette": ['red', 'green', 'blue']},'classification')

Map

Map(center=[37.75, -122.25], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(…

Yay!! 🎉 Looks like our example works. Don't party too much because there is a catch...

This workflow has several limitations particularly due to how much data you can pass from the client to the server and how large of a model ee can acutally handle. EE can only handle 40MB of data passed to the server, so if you have a lot of large decision tree strings then this will not work. Also, creating a classifier from strings has limitation (see this ee-forum discussion: https://groups.google.com/g/google-earth-engine-developers/c/lFFU1GBPzi8/m/6MewQk1FBwAJ), this is again limited by string lengths when ee creates a computation graph.

So, you can use this but know you will probably run into errors when training large models.

## Save trees to the cloud

Now we have the strings in a format that ee can use, we want to save it for later use. There is a function to export a list of tree strings to a feature collection. The feature collection will have a pro

In [45]:
user_id = geemap.ee_user_id()
user_id

'users/giswqs'

In [46]:
# specify asset id where to save trees
# be sure to change <user_name> to your ee user name
asset_id = user_id +  "/random_forest_strings_test"
asset_id

'users/giswqs/random_forest_strings_test'

In [47]:
# kick off an export process so it will be saved to the ee asset
# ml.export_trees_to_fc(trees,asset_id)

# this will kick off an export task, so wait a few minutes before moving on

In [48]:
# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)

In [49]:
# display results
# we should get the exact same results as before
Map = geemap.Map(center=(37.75,-122.25), zoom=11)

Map.addLayer(image,{"bands": ['B7', 'B5', 'B3'], "min":0.05, "max": 0.55, "gamma":1.5}, 'image')
Map.addLayer(classified, {"min": 0, "max": 2, "palette": ['red', 'green', 'blue']},'classification')

Map

Map(center=[37.75, -122.25], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(…

## Save trees locally

In [50]:
import os
out_csv = os.path.expanduser("~/Downloads/trees.csv")

In [51]:
ml.trees_to_csv(trees, out_csv)

In [52]:
another_classifier = ml.csv_to_classifier(out_csv)

In [53]:
classified = image.select(feature_names).classify(another_classifier)

In [54]:
# display results
# we should get the exact same results as before
Map = geemap.Map(center=(37.75,-122.25), zoom=11)

Map.addLayer(image,{"bands": ['B7', 'B5', 'B3'], "min":0.05, "max": 0.55, "gamma":1.5}, 'image')
Map.addLayer(classified, {"min": 0, "max": 2, "palette": ['red', 'green', 'blue']},'classification')

Map

Map(center=[37.75, -122.25], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(…